I want to analyze changes over time in the MOT GTFS feed. 

Agenda:
1. [Get data](#Get-the-data)

3. [Tidy](#Tidy-it-up)


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import partridge as ptg
from ftplib import FTP
import datetime
import re
import zipfile
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5) # set default size of plots

sns.set_style("white")
sns.set_context("talk")
sns.set_palette('Set2', 10)

## Get the data
There are two options - TransitFeeds and the workshop's S3 bucket.

In [5]:
#!aws s3 cp s3://s3.obus.hasadna.org.il/2018-04-25.zip data/gtfs_feeds/2018-04-25.zip

## Tidy it up
Again I'm using [partridge](https://github.com/remix/partridge/tree/master/partridge) for filtering on dates, and then some tidying up and transformations.

In [6]:
from gtfs_utils import *

local_tariff_path = 'data/sample/180515_tariff.zip' 

In [7]:
conn = ftp_connect()
get_ftp_file(conn, file_name = TARIFF_FILE_NAME, local_zip_path = local_tariff_path )

In [70]:


def to_timedelta(df):
    '''
    Turn time columns into timedelta dtype
    '''
    cols = ['arrival_time', 'departure_time']
    numeric = df[cols].apply(pd.to_timedelta, unit='s')
    df = df.copy()
    df[cols] = numeric
    return df

In [71]:
%time f2 = new_get_tidy_feed_df(feed, [zones])

Wall time: 2min 16s


In [72]:
f2.head()

,trip_id,departure_time,arrival_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,stop_code,route_id,direction_id,route_short_name,route_long_name,agency_id,agency_name,zone_name
0,26398526_300118,06:37:00,06:37:00,37358,1,תל אביב מרכז,32.083715,34.798247,17038,20114,0,NaN,תל אביב מרכז-תל אביב יפו<->באר שבע מרכז-באר שבע,2,רכבת ישראל,גוש דן
1,26398526_300118,06:39:00,06:39:00,37350,2,השלום,32.073346,34.793256,17046,20114,0,NaN,תל אביב מרכז-תל אביב יפו<->באר שבע מרכז-באר שבע,2,רכבת ישראל,גוש דן
2,26398526_300118,06:44:00,06:44:00,37292,3,תל אביב ההגנה,32.053978,34.784839,17104,20114,0,NaN,תל אביב מרכז-תל אביב יפו<->באר שבע מרכז-באר שבע,2,רכבת ישראל,גוש דן
3,26398526_300118,06:58:00,06:58:00,37338,4,לוד,31.945234,34.875081,17058,20114,0,NaN,תל אביב מרכז-תל אביב יפו<->באר שבע מרכז-באר שבע,2,רכבת ישראל,גוש דן
4,26398526_300118,07:22:00,07:22:00,37316,5,קרית גת,31.603526,34.777955,17080,20114,0,NaN,תל אביב מרכז-תל אביב יפו<->באר שבע מרכז-באר שבע,2,רכבת ישראל,קריית גת


In [74]:
f2.columns

Index(['trip_id', 'departure_time', 'arrival_time', 'stop_id', 'stop_sequence',
       'stop_name', 'stop_lat', 'stop_lon', 'stop_code', 'route_id',
       'direction_id', 'route_short_name', 'route_long_name', 'agency_id',
       'agency_name', 'zone_name'],
      dtype='object')

In [9]:
def get_tidy_feed_df(feed, zones):
    s = feed.stops
    r = feed.routes
    a = feed.agency
    t = (feed.trips
         # faster joins and slices with Categorical dtypes         
         .assign(route_id=lambda x: pd.Categorical(x['route_id'])))
    f = (feed.stop_times[fields['stop_times']]
         .merge(s[fields['stops']], on='stop_id')
         .merge(zones, how='left')
         .assign(zone_name=lambda x: pd.Categorical(x['zone_name']))
         .merge(t[fields['trips']], on='trip_id', how='left')
         .assign(route_id=lambda x: pd.Categorical(x['route_id']))         
         .merge(r[fields['routes']], on='route_id', how='left')
         .assign(agency_id=lambda x: pd.Categorical(x['agency_id']))
         .merge(a[fields['agency']], on='agency_id', how='left')
         .assign(agency_name=lambda x: pd.Categorical(x['agency_name']))
         .pipe(to_timedelta)
        )
    return f

In [10]:
LOCAL_ZIP_PATH = 'data/gtfs_feeds/2018-02-01.zip' 

In [11]:
feed = get_partridge_feed_by_date(LOCAL_ZIP_PATH, datetime.date(2018,2 , 1))
zones = get_zones()

In [20]:
'route_ids' in feed.routes.columns

False

In [12]:
feed.routes.shape

(6719, 7)

In [13]:
f = get_tidy_feed_df(feed, zones)

In [55]:
f.columns

Index(['trip_id', 'departure_time', 'arrival_time', 'stop_id', 'stop_sequence',
       'stop_name', 'stop_lat', 'stop_lon', 'stop_code', 'zone_name',
       'route_id', 'direction_id', 'route_short_name', 'route_long_name',
       'agency_id', 'agency_name'],
      dtype='object')

In [17]:
f[f.route_short_name.isin(['20', '26', '136'])].groupby('stop_name').route_short_name.nunique().sort_values(ascending=False)

stop_name
הרצל/עולי הגרדום                 3
כביש 57/דרך דגניה                3
צומת נווה איתמר                  3
צומת שער חפר                     3
תחנה מרכזית נתניה/רציפים         3
הרצל/מינץ                        3
תחנת רכבת                        3
צומת זיקים                       2
הגליל-גן העיר                    2
צומת השרון                       2
הגליל/6013                       2
אחוזת אוהלו                      2
הגליל/א רינה                     2
האורגים/בעלי מלאכה               2
הגליל/אל סאלם                    2
הגליל/הקישון                     2
הגליל/סאפוריה                    2
הגליל/שכונת אלרום                2
צומת חטיבת הנגב                  2
שדרות מנחם בגין/אנה פרנק         2
הגליל / דרך א סלאם               2
משרד הרישוי/הלוחמים              2
כביש 90/שדרות אליעזר קפלן        2
המעיין/פאולוס השישי              2
צומת העוגן                       2
הגליל                            2
צומת דגניה ב                     2
צומת גנות הדר                    2
הרצל/בנימי